In [ ]:
import pandas as pd
import geopandas as gp
from fiona.crs import from_epsg
import shapely
import matplotlib.pylab as plt
import csv
import dateutil
import os
from shapely.geometry import Point
%matplotlib inline

In [ ]:
def clean_columns(data,year, month):
    
    data.columns = map(lambda x: x.strip(), list(data.columns))
    data.columns = map(lambda x: x.lower(), list(data.columns))
    
    if (((year == '2015') & (month in ['07', '08', '09', '10', '11', '12'])) | (year == '2016')):
        data.drop([u'vendorid',u'store_and_fwd_flag', u'ratecodeid', u'dropoff_longitude', 
                   u'dropoff_latitude', u'passenger_count', u'fare_amount', u'extra',
                   u'mta_tax', u'tip_amount', u'tolls_amount', u'ehail_fee',
                   u'improvement_surcharge', u'total_amount', u'payment_type',
                   u'trip_type'], axis = 1, inplace = True)
    
    else: 
    
        cols = data.columns
        data.reset_index(inplace=True)
        data = data.iloc[:,:len(cols)]
        data.columns = cols
        
        if year == '2015':
            data.drop([u'vendorid',u'store_and_fwd_flag', u'ratecodeid', u'dropoff_longitude', 
                       u'dropoff_latitude', u'passenger_count', u'fare_amount', u'extra',
                       u'mta_tax', u'tip_amount', u'tolls_amount', u'ehail_fee',
                       u'improvement_surcharge', u'total_amount', u'payment_type',
                       u'trip_type'], axis = 1, inplace = True)
        
        elif year == '2014':
            data.drop([u'vendorid',u'store_and_fwd_flag', u'ratecodeid', u'dropoff_longitude', 
                       u'dropoff_latitude', u'passenger_count', u'fare_amount', u'extra',
                       u'mta_tax', u'tip_amount', u'tolls_amount', u'ehail_fee',
                       u'total_amount', u'payment_type', u'trip_type'], axis = 1, inplace = True)
               
    data.columns = ['date', 'dropoff_datetime','pickup_longitude', 'pickup_latitude', 
                    'trip_distance']
    
    return data

In [ ]:
def data_aggregator(data, tzone):
       
    data["date"] = pd.to_datetime(data["date"])
    data = data[(data['trip_distance'] <= 30) & (data['trip_distance'] > 0)]

    crs = {'init': 'epsg:4326', 'no_defs': True}
    geometry = [Point(xy) for xy in zip(data["pickup_longitude"], data["pickup_latitude"])]
    data = data.drop(["pickup_longitude", "pickup_latitude"], axis=1)
    data = gp.GeoDataFrame(data, crs=crs, geometry=geometry)

    print "Spatially joining data and taxi zones"
    data_geo = gp.sjoin(tzones, data) 
    data_geo.drop(['borough', u'OBJECTID', 'Shape_Area', 'Shape_Leng',
                   u'geometry', 'index_right', 'zone'], axis = 1, inplace = True)   
    

    data_agg = data_geo.set_index('date').groupby([pd.TimeGrouper('D'),'LocationID']).agg({'trip_distance':'mean', 
                                                                                                'dropoff_datetime': 'count'})
    data_agg.reset_index('LocationID', inplace=True)
    
    data_agg.columns = ['locationid', 'trip_distance', 'pickup_count']
    print "Done"
    return data_agg

In [ ]:
# Read data by month
def green_taxi(tzones): 
    
    
#     if not (os.path.isdir("../Data/Aggregated")):
#         print "Creating Folder Data/Aggregated"
#         os.system("mkdir ../Data/Aggregated")
        
#     year = ['2014', '2015']
#     months = ['01','02','03','04','05','06','07','08','09','10','11','12']
    year = ['2014']
    months = ['11','12']

    green = []
    for y in year:
        for m in months:
            
            print "Reading Data/Green/green_tripdata_" + y + "-" + m + ".csv"
            data = pd.read_csv("../../Data/Green/green_tripdata_" + y + "-" + m + ".csv")
            print "Cleaning columns"
            data = clean_columns(data, y, m)
            print "Aggregating data"
            data_agg = data_aggregator(data, tzones)
            data_agg.to_csv("../../Data/Aggregated/green_"+ y + "-" + m + ".csv")
            green.append(data_agg)
    
    return green

# Taxi zones geocoded data

In [ ]:
tzones = gp.read_file("../../Data/taxi_zones/taxi_zones_updated.shp")

In [ ]:
tzones.to_crs(epsg=4326, inplace=True)

In [ ]:
tzones.head(3)

In [ ]:
len(tzones.LocationID.unique())

# Run Data

In [ ]:
data = green_taxi(tzones)